## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-04-24-25 +0000,startribune,How the Minnesota Top 25 fared in Week 2 high ...,https://www.startribune.com/high-school-footba...
1,2025-09-06-04-09-04 +0000,nyt,"With ‘Ghost Bat’ Drone, Australia Gears Up for...",https://www.nytimes.com/2025/09/06/world/austr...
2,2025-09-06-04-03-34 +0000,nypost,102-year-old man sets Guinness World Record as...,https://nypost.com/2025/09/06/world-news/102-y...
3,2025-09-06-04-01-32 +0000,nyt,Food Delivery Drivers Feel Strain of Italy’s B...,https://www.nytimes.com/2025/09/06/world/europ...
4,2025-09-06-04-01-28 +0000,nyt,Inside Iran After the 12-Day War,https://www.nytimes.com/video/world/middleeast...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
104,trump,50
144,will,14
12,new,14
11,up,12
111,report,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,94
69,2025-09-05-22-25-26 +0000,nyt,A Second Weak Jobs Report Challenges Trump’s C...,https://www.nytimes.com/2025/09/05/us/politics...,87
24,2025-09-06-01-39-40 +0000,nypost,Kilmar Abrego Garcia faces deportation to tiny...,https://nypost.com/2025/09/05/us-news/kilmar-a...,86
246,2025-09-05-13-39-23 +0000,nypost,US deploying 10 fighter jets to Puerto Rico in...,https://nypost.com/2025/09/05/us-news/us-deplo...,85
55,2025-09-05-23-05-39 +0000,nypost,Barron Trump ‘doing a semester’ at different N...,https://nypost.com/2025/09/05/us-news/barron-t...,83


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,94,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
132,62,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
51,54,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
296,52,2025-09-05-09-42-52 +0000,nypost,Putin warns Western troops deployed to Ukraine...,https://nypost.com/2025/09/05/world-news/russi...
113,47,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
28,46,2025-09-06-01-20-10 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
179,44,2025-09-05-17-33-52 +0000,latimes,CBS' 'Face the Nation' will no longer edit tap...,https://www.latimes.com/entertainment-arts/bus...
215,40,2025-09-05-15-39-10 +0000,nyt,"Angela Rayner, UK Deputy Prime Minister, Resig...",https://www.nytimes.com/2025/09/05/world/europ...
58,39,2025-09-05-22-59-37 +0000,nypost,Andrew Cuomo touts charter school support — wh...,https://nypost.com/2025/09/05/us-news/andrew-c...
18,36,2025-09-06-02-17-09 +0000,nypost,Ex-FBI official tipped off Chinese firm doing ...,https://nypost.com/2025/09/05/us-news/ex-fbi-o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
